# <strong>Post-processing Finite Element Analysis in Python: Use Case.</strong>

In this session, we will explore a simple use case to demonstrate the advantages of using Python for post-processing finite element analysis (FEA) results. We will see how easy and efficient it is to extract the mesh structure, access results, manipulate data, combine load cases, generate a CSV file for plotting, and visualize the post-processed results.

**Table of Contents:**
1. [Why Python](#1-why-python-)
2. [Tools Introduction](#2-tools-introduction-)
3. [Working with the Mesh](#3-working-with-the-mesh-️)
4. [The Results](#4-the-results-)
5. [Writing the Results](#5-writing-the-results)

## 1. Why-Python 🐍

Python is a user-friendly programming language with a gentle learning curve. This means that anyone, with minimal effort, can learn the basics and automate tasks with just a few lines of code. The Python community is extensive, and there is a wealth of information available on the internet. Additionally, tools like ChatGPT can assist in generating high-quality Python code with clear instructions.

For engineers, Python is an invaluable tool that helps improve productivity and eliminates repetitive, tedious tasks.

Compared to batch files, CMD scripts, or PowerShell (PS1) scripts, Python is easier to use for automating tasks within the operating system. For data analysis, Python is significantly more powerful and intuitive than Excel and VBA macros.

## 2. Tools Introduction 🔧

We are using VS Code to work with Jupyter Notebooks. As you will see, Notebooks are useful for explain code as it allow to run Python code in a cool format document. The python version is the 3.11.7 as you can see in the top right corner. As external package we are using [NaxToPy](<https://pypi.org/project/NaxToPy/>), a package that will allow as to open and load imput mesh files (.inp, .bdf, .fem) and binary result files (.op2, .odb, .h3d, .xdb, .rst). The commands and almost all the code is the same for every solver supported (Nastran, Abaqus, Optistruct and Ansys). For using this package it is necessary to have [NaxTo](<https://www.idaerosolutions.com/Home/NaxTo>) instaled. NaxTo is a trusted post-processing suite, validated by Tier 1 aeronautical companies and in use since 2017. It offers seamless integration with Excel, Python, Word, and its dedicated visualizer, NaxToView.

There are other Python packages for post-processing FEA results files, but they are not as easy or straightforward to use as NaxToPy, as we will see. Once installed, using NaxToPy is as simple as importing it:

In [1]:
import NaxToPy as n2p

n2p.__version__

'2.1.2'

## 3. Working with the Mesh ✈️

In order to work with the mesh is as easy as use the coman `load_model()` and the path to the mesh. A python object is created with all the information of the model. Modern IDEs and editors (PyCharm, VSCode, Spyder, etc) allow us to debug the variables and see what there is inside the objects.

In [2]:
path = r"C:\Webinar\model\SUBCASE_17500.dat"
model = n2p.load_model(path)
model

Objects in Python have methods that allow us to query information. These methods are suggested by modern IDEs and editors, and their details can be accessed simply by hovering the cursor over the method.

In [5]:
model.get_elements((39800008, '0'))

N2PElement(39800008, '0')

The simple and powerful syntax of Python allows us to write efficient code, sometimes in just a single line.

In [7]:

cquads = [ele for ele in model.get_elements() if ele.TypeElement == "CQUAD4"]
cquads[12].TypeElement

'CQUAD4'

## 4. The Results 📊

Loading results into the mesh is straightforward. It only requires a single function.

In [8]:
print(model.LoadCases)

result_files = [
    r"C:\Webinar\model\subcase_17500.op2",
    r"C:\Webinar\model\subcase_17501.op2",
    r"C:\Webinar\model\subcase_17502.op2",
    r"C:\Webinar\model\subcase_17503.op2",
    r"C:\Webinar\model\subcase_17504.op2"
]

model.import_results_from_files(result_files)

print(model.LoadCases)

[]
[N2PLoadCase(17500: '2DP'), N2PLoadCase(17501: ' PRESSURE+MECHANICAL'), N2PLoadCase(17502: ' PRESSURE+MECHANICAL+TEMPERATURE COLD'), N2PLoadCase(17503: ' PRESSURE+MECHANICAL+TEMPERATURE HOT'), N2PLoadCase(17504: ' MECHANICAL+TEMPERATURECOLD')]


Imagine that the first load case is thermal, and the others are mechanical load cases. These can be combined using a formula defined as a string.

In [9]:
for lc in model.LoadCases[1:]:
    formula = f"1.5*<LC{lc.ID}:FR1>+<LC17500:FR1>"
    model.new_derived_loadcase(f"{lc.ID}+Thermal", formula)


print(model.LoadCases)

[N2PLoadCase(17500: '2DP'), N2PLoadCase(17501: ' PRESSURE+MECHANICAL'), N2PLoadCase(17502: ' PRESSURE+MECHANICAL+TEMPERATURE COLD'), N2PLoadCase(17503: ' PRESSURE+MECHANICAL+TEMPERATURE HOT'), N2PLoadCase(17504: ' MECHANICAL+TEMPERATURECOLD'), N2PLoadCase(-1: '17501+Thermal'), N2PLoadCase(-2: '17502+Thermal'), N2PLoadCase(-3: '17503+Thermal'), N2PLoadCase(-4: '17504+Thermal')]


Next, the envelope load case is generated.

In [11]:
formula_env = ",".join([f"<LC{lc.ID}:FR0>" for lc in model.LoadCases if lc.TypeLC == "DERIVED"])
formula_env

env_ct = model.new_envelope_loadcase("ENVELOPE", formula_env, "ExtremeMax", "ByContour")
env_lc = model.new_envelope_loadcase("ENVELOPE LC", formula_env, "ExtremeMax", "ByLoadCaseID")

print(model.LoadCases)

[N2PLoadCase(17500: '2DP'), N2PLoadCase(17501: ' PRESSURE+MECHANICAL'), N2PLoadCase(17502: ' PRESSURE+MECHANICAL+TEMPERATURE COLD'), N2PLoadCase(17503: ' PRESSURE+MECHANICAL+TEMPERATURE HOT'), N2PLoadCase(17504: ' MECHANICAL+TEMPERATURECOLD'), N2PLoadCase(-1: '17501+Thermal'), N2PLoadCase(-2: '17502+Thermal'), N2PLoadCase(-3: '17503+Thermal'), N2PLoadCase(-4: '17504+Thermal'), N2PLoadCase(-5: 'ENVELOPE'), N2PLoadCase(-6: 'ENVELOPE LC')]


At this point, the results have not been extracted. This can be done using several methods:

### 4.1 Simple Results Extraction 📉

In [15]:
fx = env_ct.get_result("FORCES").get_component("FX").get_result_ndarray()[0][:len(model.get_elements())]
fx

fx_lc = env_lc.get_result("FORCES").get_component("FX").get_result_ndarray()[0][:len(model.get_elements())]
fx_lc

array([-2., -2., -4., ..., -3., -2., -3.])

### 4.2 Parallel Results Extraction 📈

In [17]:
fx_parallel = model.get_result_by_LCs_Incr([(env_lc, env_lc.ActiveN2PIncrement)], "FORCES", "FX")
fx_parallel

{(-6, 0): array([-2., -2., -4., ..., nan, nan, nan])}

### 4.3 Working with the Results 🔨

In [ ]:
cquads_internal_ids = [ele.InternalID for ele in cquads]
cquads_internal_ids

fx_cqudas = fx[cquads_internal_ids]
fx_lc_cquads = fx_lc[cquads_internal_ids]

N2PElement(41900071, '0')

## 5. Writing the Processed Results 📝

The results can be printed in the format specified by the user program. In this case, we will use the format of the NaxTo visualizer. The output format is a CSV file with two mandatory columns: "ID_E" and "PARTS_E". Therefore, we need to prepare the CQUAD4 IDs, the CQUAD4 Part IDs, and two result columns: the FX envelope and the load case in which the maximum value was found.

In [23]:
import csv

id_e = [ele.ID for ele in model.get_elements()]
part_e = [ele.PartID for ele in model.get_elements()]

with open(".\\fx_cquad_envelope.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["ID_E", "PARTS_E", "FX", "LoadCase"])  # Write header
    writer.writerows(zip(id_e, part_e, fx_cqudas, fx_lc_cquads))  # Write data rows